In [20]:
import pandas as pd
from pathlib import Path
from loguru import logger
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score

from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

import joblib
from scipy.stats import pearsonr, spearmanr
from matplotlib import pyplot as plt

In [21]:
def load_and_merge_dye_data(data_dir: Path) -> pd.DataFrame:
    """加载和合并荧光染料数据集"""
    fluor_data_dir = data_dir / "dyes"
    
    # 检查目录是否存在
    if not fluor_data_dir.exists() or not fluor_data_dir.is_dir():
        logger.error(f"目录 {fluor_data_dir} 不存在或不是有效的目录。")
        return pd.DataFrame()  # 返回空 DataFrame

    # 文件名列表
    files = [
        "Dataset_Consolidation_canonicalized.csv",
        "Dataset_Cyanine_canonicalized.csv",
        "Dataset_Xanthene_canonicalized.csv",
    ]

    # 检查文件是否存在
    missing_files = [f for f in files if not (fluor_data_dir / f).exists()]
    if missing_files:
        logger.error(f"以下文件缺失: {', '.join(missing_files)}")
        return pd.DataFrame()  # 返回空 DataFrame

    # 加载数据
    dfs = []
    for file in files:
        try:
            df = pd.read_csv(fluor_data_dir / file)
            dfs.append(df)
            logger.info(f"成功加载文件: {file}")
        except Exception as e:
            logger.error(f"加载文件 {file} 时出错: {e}")
            return pd.DataFrame()  # 出错时返回空 DataFrame

    # 合并数据并去重
    merged_df = pd.concat(dfs, ignore_index=True).reset_index(drop=True)
    n_stokes_shift = 0
    n_emission_absorption = 0
    for index, row in merged_df.iterrows():
        if row["stokes_shift"]:
            n_stokes_shift += 1
            continue
        elif row["emission"] and row["absorption"]:
            n_emission_absorption += 1
            merged_df.loc[index, "stokes_shift"] = row["emission"] - row["absorption"]
        else:
            merged_df.loc[index, "stokes_shift"] = None
            
    logger.info(f"Stokes shift 计算成功: {n_stokes_shift} 条数据，{n_emission_absorption} 条数据需要计算。")
    # 选择所需列
    merged_df = merged_df[["smiles", "absorption","emission","stokes_shift", "quantum_yield"]]
    merged_df.dropna(inplace=True)
    merged_df = merged_df.drop_duplicates(subset=["smiles"], keep="first")
    
    # 重置索引
    merged_df.reset_index(drop=True, inplace=True)
    
    # 输出加载的数据量
    logger.info(f"已加载 {len(merged_df)} 条唯一 SMILES 数据。")
    return merged_df

In [22]:
data_dir = Path("../data")
df = load_and_merge_dye_data(data_dir)

2025-06-14 06:32:11.532 | INFO     | __main__:load_and_merge_dye_data:29 - 成功加载文件: Dataset_Consolidation_canonicalized.csv
2025-06-14 06:32:11.539 | INFO     | __main__:load_and_merge_dye_data:29 - 成功加载文件: Dataset_Cyanine_canonicalized.csv
2025-06-14 06:32:11.545 | INFO     | __main__:load_and_merge_dye_data:29 - 成功加载文件: Dataset_Xanthene_canonicalized.csv


2025-06-14 06:32:12.436 | INFO     | __main__:load_and_merge_dye_data:48 - Stokes shift 计算成功: 39398 条数据，0 条数据需要计算。
2025-06-14 06:32:12.445 | INFO     | __main__:load_and_merge_dye_data:58 - 已加载 6703 条唯一 SMILES 数据。


In [23]:
df.describe()

,absorption,emission,stokes_shift,quantum_yield
count,6703.000000,6703.000000,6703.000000,6703.000000
mean,453.898255,530.709085,76.811428,0.350122
std,117.823913,107.912885,50.847197,0.288046
min,222.000000,247.000000,1.000000,0.000040
25%,367.000000,447.000000,33.000000,0.084000
50%,423.000000,521.000000,70.000000,0.290000
75%,519.000000,598.000000,111.000000,0.600000
max,943.000000,1097.000000,325.000000,1.000000


In [24]:
def smiles_to_morgan_fingerprint(smiles: str, radius: int = 2, n_bits: int = 2048, return_bitinfo=False):
    """
    将 SMILES 转换为 Morgan 指纹向量。
    支持返回 bitInfo，方便后续解释性分析。
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")
        bitInfo = {}
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits, bitInfo=bitInfo)
        arr = np.zeros((n_bits,), dtype=int)
        AllChem.DataStructs.ConvertToNumpyArray(fingerprint, arr)
        if return_bitinfo:
            return arr, bitInfo
        else:
            return arr
    except Exception as e:
        logger.warning(f"Invalid SMILES '{smiles}': {e}")
        if return_bitinfo:
            return np.zeros(n_bits), {}
        else:
            return np.zeros(n_bits)

def prepare_features(df: pd.DataFrame, radius=2, n_bits=2048):
    """
    将整个 dataframe 的 SMILES 列批量转化为特征矩阵，并返回 bitInfo 列表
    """
    fingerprints = []
    bitinfos = []
    for smiles in df["smiles"]:
        fp, bitInfo = smiles_to_morgan_fingerprint(smiles, radius, n_bits, return_bitinfo=True)
        fingerprints.append(fp)
        bitinfos.append(bitInfo)
    X = pd.DataFrame(fingerprints)
    return X, bitinfos

def train_lightgbm_model(df: pd.DataFrame, test_size=0.1, random_state=42):
    """
    训练多输出 LightGBM 模型，返回模型、评估指标、真实值和预测值
    """
    logger.info("Converting SMILES to Morgan Fingerprints...")
    X, bitinfos = prepare_features(df)
    
    targets = ["absorption", "emission", "stokes_shift", "quantum_yield"]
    y = df[targets]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    logger.info("Training LightGBM model...")
    lgb_model = lgb.LGBMRegressor(objective="regression", random_state=random_state)
    model = MultiOutputRegressor(lgb_model)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    metrics = {}
    for i, target in enumerate(targets):
        mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
        metrics[target] = {"MSE": mse, "R2": r2, "RMSE": rmse}
        logger.info(f"{target} - MSE: {mse:.4f}, R2: {r2:.4f}, RMSE: {rmse:.4f}")
    return model, metrics, y_test, y_pred, X_test, bitinfos

In [19]:
model, metrics, y_test, y_pred, X_test, bitinfos = train_lightgbm_model(df)

2025-06-14 06:31:44.617 | INFO     | __main__:train_lightgbm_model:42 - Converting SMILES to Morgan Fingerprints...
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator
[06:31:44] DEPRECATION WARNING: please use MorganGenerator

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3242
[LightGBM] [Info] Number of data points in the train set: 6032, number of used features: 1621
[LightGBM] [Info] Start training from score 455.088031
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023462 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3242
[LightGBM] [Info] Number of data points in the train set: 6032, number of used features: 1621
[LightGBM] [Info] Start training from score 531.524038
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027046 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

2025-06-14 06:32:11.205 | INFO     | __main__:train_lightgbm_model:65 - absorption - MSE: 1456.0986, R2: 0.8839, RMSE: 38.1589
2025-06-14 06:32:11.207 | INFO     | __main__:train_lightgbm_model:65 - emission - MSE: 1847.3987, R2: 0.8274, RMSE: 42.9814
2025-06-14 06:32:11.209 | INFO     | __main__:train_lightgbm_model:65 - stokes_shift - MSE: 881.0986, R2: 0.6629, RMSE: 29.6833
2025-06-14 06:32:11.210 | INFO     | __main__:train_lightgbm_model:65 - quantum_yield - MSE: 0.0444, R2: 0.4833, RMSE: 0.2107


In [ ]:
with open("../dyeles/resources/lightgbm_shapley.pkl", "wb") as f:
    joblib.dump(model, f)

In [ ]:
def plot_r2_scatter_separate(y_true: pd.DataFrame, y_pred: np.ndarray, targets=None, dpi=600):
    """
    分别为每个目标变量绘制独立的 R² 散点图
    """
    if targets is None:
        targets = y_true.columns.tolist()
    
    for i, target in enumerate(targets):
        plt.figure(figsize=(4, 4), dpi=dpi)
        plt.scatter(y_pred[:, i], y_true[target], alpha=0.5)
        r2 = r2_score(y_true[target], y_pred[:, i])
        plt.title(f"{target} (R² = {r2:.2f})")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.grid(True)
        
        # 设置不同目标变量的坐标轴范围
        if target == "absorption" or target == "emission":
            low = 200
            high = 1001
            plt.xlim(low, high)
            plt.ylim(low, high)
            plt.xticks(np.arange(low, high, 100))
            plt.yticks(np.arange(low, high, 100))
        elif target == "stokes_shift":
            low = 0
            high = 300
            plt.xlim(low, high)
            plt.ylim(low, high)
            plt.xticks(np.arange(low, high, 50))
            plt.yticks(np.arange(low, high, 50))
        elif target == "quantum_yield":
            low = 0
            high = 1.1
            plt.xlim(low, high)
            plt.ylim(low, high)
            plt.xticks(np.arange(low, high, 0.2))
            plt.yticks(np.arange(low, high, 0.2))
        else:
            # 如果未定义范围，使用默认的范围
            low = np.min(y_true[target].min(), y_pred[:, i].min())
            high = np.max(y_true[target].max(), y_pred[:, i].max())
        
        
        plt.tight_layout()
        plt.show()

targets = ["absorption", "emission", "stokes_shift", "quantum_yield"]
plot_r2_scatter_separate(y_test, y_pred, targets=targets)

## Shapley

In [30]:
import shap
from rdkit.Chem import Draw

In [28]:
# 取 absorption 模型 (MultiOutputRegressor 的第一个 estimator_)
absorption_model = model.estimators_[0]
# emission_model = model.estimators_[1]
# stock_model = model.estimators_[2]
# quantum_model = model.estimators_[3]

# 计算 SHAP 值
explainer = shap.TreeExplainer(absorption_model)
X, bitinfos = prepare_features(df)
shap_values = explainer.shap_values(X)

[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerator
[06:35:44] DEPRECATION WARNING: please use MorganGenerat

In [35]:
# 创建保存目录
cache_dir = Path(".cache")
cache_dir.mkdir(exist_ok=True)

# 以第一个分子为例
sample_idx = 2

# 取出该样本的 SHAP 值向量
sample_shap = shap_values[sample_idx]

# 取出最重要的前 N 个 bits
N = 5
important_bits = np.argsort(np.abs(sample_shap))[::-1][:N]

# 取出该分子的 smiles 和 mol
smiles_sample = df.iloc[sample_idx]["smiles"]
mol = Chem.MolFromSmiles(smiles_sample)

# 保存整体分子图片
img = Draw.MolToImage(mol, size=(200, 200))
img.save(cache_dir / f"mol_{sample_idx}.png")

# 取出该分子的 bitinfo
bitinfo_sample = bitinfos[sample_idx]

# 映射每个重要 bit 回 fragment
for bit in important_bits:
    shap_value = sample_shap[bit]
    print(f"Bit {bit}, SHAP value: {shap_value:.4f}")

    if bit in bitinfo_sample:
        for i, (atom_idx, radius) in enumerate(bitinfo_sample[bit]):
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom_idx)
            submol = Chem.PathToSubmol(mol, env)
            smi = Chem.MolToSmiles(submol)
            print(f"    Fragment {i+1}: {smi}")

            # 可视化并保存片段
            frag_img = Draw.MolToImage(submol, size=(200, 200))
            frag_img.save(cache_dir / f"fragment_{sample_idx}_bit{bit}_{i+1}.png")
    else:
        print(f"    Bit {bit} not found in bitInfo.")

Bit 1100, SHAP value: 24.2581
    Fragment 1: 
Bit 463, SHAP value: -20.5598
    Bit 463 not found in bitInfo.
Bit 1724, SHAP value: 11.3150
    Fragment 1: cc(c)n
Bit 960, SHAP value: 9.0634
    Fragment 1: cccc(c)N
    Fragment 2: cccc(c)N
    Fragment 3: cccc(c)N
    Fragment 4: cccc(c)N
    Fragment 5: cccc(c)N
    Fragment 6: cccc(c)N
Bit 1964, SHAP value: -8.1209
    Fragment 1: ccc(cc)-c(c)c
